In [4]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from pyod.models.iforest import IForest
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import OneSidedSelection
import os
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pyod.models.pca import PCA
from sklearn.mixture import GaussianMixture
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import GridSearchCV

import os, sys, pickle
import argparse, sys

import seaborn as sns
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')



# parser = argparse.ArgumentParser()
# parser.add_argument('-label', help=' : Please set the label') 
# args = parser.parse_args()


df = pd.read_csv('data/modified_dataset.csv')


# def remove_outliers(df, column_name, factor=1.5):
#     Q1 = df[column_name].quantile(0.25)
#     Q3 = df[column_name].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - factor * IQR
#     upper_bound = Q3 + factor * IQR
#     df_filtered = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
#     return df_filtered

# # 모든 칼럼에 대해 이상치 제거 실행
# for column_name in df.columns:
#     df = remove_outliers(df, column_name)







df_stroke_0 = df[df['Cancer'] == 0]
df_stroke_1 = df[df['Cancer'] == 1]



#train, test data processing 1,2,3,4중 선택

#1.정상으로만 학습, 테스트는 정상과 이상 반반
answer_label = 'Cancer'
# X = df[df.columns.difference([answer_label])]
# df_normal = df[df[answer_label] == 0]
# df_abnormal = df[df[answer_label] == 1]
# test_normal_df = df_normal.sample(n=len(df_stroke_1), random_state = 0)
# test_df = pd.concat([df_abnormal, test_normal_df])
# X_test = test_df[test_df.columns.difference([answer_label])]
# y_test = test_df[answer_label]
# train_df = df_normal.drop(test_normal_df.index)
# X_train = train_df[train_df.columns.difference([answer_label])]
# y_train = train_df[answer_label]

#그냥데이터
# X = df.drop(answer_label,axis=1)
# y = df[answer_label]
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

#3.Oversampling by Smote
overs = SMOTE(random_state=1)
X = df.drop(answer_label,axis=1)
y = df[answer_label]
X_os, y_os = overs.fit_resample(X,y)
X_train,X_test,y_train,y_test = train_test_split(X_os,y_os,test_size=0.3,random_state=1)



In [5]:
def main_task(contamination):
    gde = EllipticEnvelope(contamination=contamination)
    gde.fit(X_train)
    GDE_test_pred = gde.predict(X_test)
    GDE_test_pred = pd.DataFrame(GDE_test_pred)
    GDE_test_pred = GDE_test_pred.replace({-1:1, 1:0})
    return round(f1_score(y_test, GDE_test_pred),3)

In [6]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

def optuna_main(trial: optuna.Trial):
    contamination = trial.suggest_float('contamination', 0.01, 0.5, step = 0.02)
    f1_score = main_task(contamination=contamination)

    print('<f1_score:', f1_score,'>')
    return f1_score

study = optuna.create_study(direction='maximize')
study.optimize(optuna_main, n_trials=100)
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_params : {study.best_trial.params}')

[I 2023-11-08 15:40:22,297] A new study created in memory with name: no-name-fb985a8e-eb12-4a5a-98ab-370b9afc37bd
[I 2023-11-08 15:40:25,993] Trial 0 finished with value: 0.269 and parameters: {'contamination': 0.25}. Best is trial 0 with value: 0.269.


<f1_score: 0.269 >


[I 2023-11-08 15:40:29,270] Trial 1 finished with value: 0.444 and parameters: {'contamination': 0.49}. Best is trial 1 with value: 0.444.


<f1_score: 0.444 >


[I 2023-11-08 15:40:32,402] Trial 2 finished with value: 0.289 and parameters: {'contamination': 0.29000000000000004}. Best is trial 1 with value: 0.444.


...

<f1_score: 0.313 >


[I 2023-11-08 15:41:30,122] Trial 20 finished with value: 0.444 and parameters: {'contamination': 0.49}. Best is trial 1 with value: 0.444.


<f1_score: 0.444 >


[W 2023-11-08 15:41:33,003] Trial 21 failed with parameters: {'contamination': 0.49} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/mohee/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_16895/3811043015.py", line 7, in optuna_main
    f1_score = main_task(contamination=contamination)
  File "/tmp/ipykernel_16895/2520945150.py", line 3, in main_task
    gde.fit(X_train)
  File "/home/mohee/.local/lib/python3.10/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/mohee/.local/lib/python3.10/site-packages/sklearn/covariance/_elliptic_envelope.py", line 184, in fit
    super().fit(X)
  File "/home/mohee/.local/lib/python3.10/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/mohee/.local/lib/python3.10/site-pa

KeyboardInterrupt: 

In [22]:
#결과
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_paramse : {study.best_params}')

study.best_trial.value : 0.96
study.best_paramse : {'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'brute', 'metric': 'manhattan'}
